# Load data

In [1]:
import os
import gc
import pandas as pd

import pickle
from collections import Counter
from tqdm import tqdm
import numpy as np
import random

from sklearn import metrics

import json
import ast

In [2]:
with open(f'{os.getcwd()}/data/author_to_subID_to_Values_withCommValueTradeoff.pkl', 'rb') as f:
    author_to_subID_to_Values = pickle.load(f)
# df_values_all = pd.read_csv(f'{os.getcwd()}/data/data_active_authorExt_withAbstValues.tsv', sep='\t')
df_values = pd.read_csv(f'{os.getcwd()}/data/data_active_authorExt_withAbstValues.tsv', sep='\t')

In [4]:
df_values

,subID,title,selftext,author,comment,judgment,situ_val,comm_val,situ_val_patterns,situ_val_pattern_cats,comm_val_patterns,comm_val_pattern_cats,id
0,k8zq74,not doing my stepchildren's hair?,"Hello, I'm 30F married to my partner 51M and h...",NoApollonia,The children are both old enough to choose if ...,1,"[""BM's value of appearance and expectation for...","```json\n{\n ""Conflicting Values"": [\n ...",Social Appearance and Judgment vs. Family Dyna...,Societal Norms and Individual Impact vs. Perso...,Self-Expression and Comfort > Parenting Author...,Themes of Identity Exploration > Parenting Dec...,k8zq74###NoApollonia
1,k93tjn,insulting my wife in front of our son.,Growing up I had faced a few key issues that I...,NoApollonia,Don't disrespect your wife in front of your so...,0,['Personal Achievement vs. Academic Excellence...,"```json\n{\n ""Conflicting Values"": [\n ...",Personal Career Satisfaction vs. Academic Pres...,Career Development and Satisfaction Themes vs....,Academic Pressure and Expectations > Personal ...,Educational Themes and Responsibilities > Care...,k93tjn###NoApollonia
2,k96h8e,telling my daughter to suck it up?,I (43M) and my wife (45F) have three kids: 12F...,NoApollonia,While I agree the two sisters should share a r...,0,"['Parental authority vs. child autonomy', 'Fin...","```json\n{\n ""Conflicting Values"": [\n ...",Parenting Authority Dynamics vs. Daughter's Au...,Parenting Decision-Making and Interactions vs....,Equity in Resource Distribution > Balancing In...,Fairness and Equity Themes > Relationship Dyna...,k96h8e###NoApollonia
3,k9o2vg,refusing to help my husband financially,So firstly English isn't my first language so ...,WebbieVanderquack,"You don't have to give him 1K, especially if y...",0,"['Financial dependence vs. independence', 'Sup...","```json\n{\n ""Conflicting Values"": [\n ...",Economic Pressures and Constraints vs. Indepen...,Financial Awareness and Responsibility Analysi...,Types of Support > Future Security and Stabili...,Exploring Support Dynamics in Text > Future-Or...,k9o2vg###WebbieVanderquack
4,k9om9o,telling my friend to take off her makeup?,I'm new to this but just a few minutes ago me ...,WebbieVanderquack,both for not using punctuation and for telling...,0,['Concern for health and well-being vs. Respec...,"```json\n{\n ""Conflicting Values"": [\n ...",Health and Safety Concerns vs. Respect for Age...,Health-Safety Awareness and Education vs. Pers...,disrecollectful and insensitivity > Friendship...,Emotional and Social Sensitivity Issues > Expl...,k9om9o###WebbieVanderquack
...,...,...,...,...,...,...,...,...,...,...,...,...,...
53275,c0dsnr,telling my poor friend that if she stopped spe...,I’ve been friends with Melanie for 3 years (we...,annshine,"Yeah I'm with you, and wonder if its because i...",0,['Responsibility and financial literacy vs. en...,"{\n 'Conflicting Values': [\n ""Pragm...",Budgeting and Financial Management vs. Emotion...,Financial Awareness and Responsibility Analysi...,Budgeting and Financial Management > Emotional...,Financial Awareness and Responsibility Analysi...,c0dsnr###annshine
53276,c0i3oz,Introducing my GF as a Model?,So I have been dating this Girl for awhile now...,annshine,because why did you need to tell strangers how...,0,['Personal identity vs. professional identity'...,"{\n 'Conflicting Values': [\n ""Perso...",Self-Identity and Acceptance vs. Professional ...,Themes of Identity Exploration vs. Expectation...,Self-Identity and Acceptance > Professional St...,Themes of Identity Exploration > Expectations ...,c0i3oz###annshine
53277,c0i3oz,Introducing my GF as a Model?,So I have been dating this Girl for awhile now...,ZenDendou,&amp;#x200B; Pay attention next time. If she d...,0,['Personal identity vs. professional identity'...,"```json\n{\n ""Conflicting Values"": [\n ...",Self-Identity and Acceptance vs. Professional ...,Themes of Identity Exploration vs. Expectation...,Individual Attachments and Choices > Social Pe...,Interpersonal

In [10]:
# df = pd.read_csv(f"{os.getcwd()}/data/data_active.tsv", sep="\t")
df_ext = pd.read_csv(f"{os.getcwd()}/data/data_active_authorExt.tsv", sep="\t")

In [4]:
df_withValues = pd.read_csv(f'{os.getcwd()}/../AbstractiveSubjectivity/gpt_answers/AITA/situation_dimensions.tsv', sep='\t')
df_schwartz = pd.read_csv(f'{os.getcwd()}/../AbstractiveSubjectivity/data/AITA/directComms_withValues.tsv', sep='\t')

In [5]:
import openai
from openai import OpenAI
from tenacity import (
    retry,
    stop_after_attempt,
    wait_random_exponential,
)

# Load your API key from an environment variable or secret management service
openai.api_key = os.getenv("OPENAI_API_KEY")
client = OpenAI()

@retry(wait=wait_random_exponential(min=1, max=60), stop=stop_after_attempt(6))
def completion_with_backoff(**kwargs):
    return client.chat.completions.create(**kwargs)

# For existing data--prompt again for confidence interval

- Run multiple times and compute the standard deviation (similar to https://huggingface.co/blog/wolfram/llm-comparison-test-2024-12-04)
- Essentially, we will search relevant items by
    - Situation description (title + selftext)
    - Competing Schwartz's values (or most salient Schwartz's values)
        - Most salient ones (2~3 items, e.g. "A, B, C"->vector_1)
    - LLM-annotated values
        - Overall aggregated vector (e.g. "A vs. B, C vs. D"->vector_1)
        - Competing Value-by-value (e.g. "A vs. B"->vector_1, "C vs. D"->vector_2)
    - Clustered LLM-annotated values
        - Overall aggregated vector (e.g. "A vs. B, C vs. D"->vector_1)
        - Competing Value-by-value (e.g. "A vs. B"->vector_1, "C vs. D"->vector_2)
- Also, in the few-shot examples, we include Subjective Ground using:
    - Judgment
    - Comment
    - Summary of comment (rules-of-thumb generated from comment)
    - Schwartz's Value
    - LLM-generated Value
- Some techniques to boost up the performance
    - CoT prompting
    - Reranking
    - Prompt compression (summary)

## Generate embeddings

In [86]:
import tiktoken
encoding = tiktoken.encoding_for_model("gpt-4o")

def slice_text_list(text_list, thr=8000):
    cnt = 0
    final_slice, sub_slice = [], []
    for text in text_list:
        encoded = encoding.encode(text)
        if cnt + len(encoded) < thr:
            sub_slice.append(text)
            cnt += len(encoded)
        else:
            final_slice.append(sub_slice)
            sub_slice = [text]
            cnt = len(encoded)
    final_slice.append(sub_slice)
    return final_slice

def normalize_l2(x):
    x = np.array(x)
    if x.ndim == 1:
        norm = np.linalg.norm(x)
        if norm == 0:
            return x
        return x / norm
    else:
        norm = np.linalg.norm(x, 2, axis=1, keepdims=True)
        return np.where(norm == 0, x, x / norm)

def process_dimensions(input_text):
    text_list = input_text.split('\n')
    return_list = []
    for text in text_list:
        if text.strip() == '':
            continue
        text = text.replace('[Answer]','').replace('**','').replace('- ','').strip()
        if text[0].isdigit():
            text = text[1:]
        if text[0]=='.':
            text = text[1:]
        return_list.append(text.strip())
    return '\n'.join(return_list)

def get_embeddings(all_texts, emb_model="text-embedding-3-large"):
    all_texts_sliced = slice_text_list(all_texts)
    all_embeddings = []
    for texts in tqdm(all_texts_sliced):
        results = []
        prompt_texts = [text.replace('\n',' ') for text in texts]

        response = client.embeddings.create(input=prompt_texts,model=emb_model)
        
        all_embeddings += [item.embedding for item in response.data]
    return all_embeddings

### Situation embedding

In [37]:
subID_to_text = {}
for idx, row in df_withValues.iterrows():
    subID_to_text[row['subID']] = [row['title'], row['selftext']]

all_subIDs, all_situations = [], []
for key in subID_to_text:
    all_subIDs.append(key)
    all_situations.append(subID_to_text[key][0]+' '+subID_to_text[key][1])
    
all_situation_embeddings = get_embeddings(all_situations)
assert len(all_subIDs)==len(all_situations)==len(all_situation_embeddings)

situation_reduced_emb = [normalize_l2(emb[:256]) for emb in all_situation_embeddings]

situation_data = {'ID':all_subIDs, 'text':all_situations, 'embedding':all_situation_embeddings, 'embedding_reduced':situation_reduced_emb}
with open(f'{os.getcwd()}/embedding/situation_text-embedding-3-large.pkl', 'wb') as f:
    pickle.dump(situation_data, f)

100%|██████████████████████████████████████████████████████████████████████| 1136/1136 [13:13<00:00,  1.43it/s]


### Situation value embedding (aggregated)

In [ ]:
all_subIDs, all_situationValues = [], []
for idx, row in df_withValues.iterrows():
    all_subIDs.append(row['subID'])
    all_situationValues.append(process_dimensions(row['dimensions']))
    
all_situValue_embeddings = get_embeddings(all_situationValues)
assert len(all_subIDs)==len(all_situationValues)==len(all_situValue_embeddings)

situValue_reduced_emb = [normalize_l2(emb[:256]) for emb in all_situValue_embeddings]

situationValue_data = {'ID':all_subIDs, 'text':all_situationValues, 'embedding':all_situValue_embeddings, 'embedding_reduced':situValue_reduced_emb}
with open(f'{os.getcwd()}/embedding/situationValueAgg_text-embedding-3-large.pkl', 'wb') as f:
    pickle.dump(situationValue_data, f)

### Situation value embedding (each competing pair)

In [ ]:
all_subIDs, all_situationValues = [], []
for idx, row in df_withValues.iterrows():
    processed_value_dimensions = process_dimensions(row['dimensions']).split('\n')
    for i, item in enumerate(processed_value_dimensions):
        all_subIDs.append(row['subID']+'#'+str(i))
        all_situationValues.append(item)
    
all_situValue_embeddings = get_embeddings(all_situationValues)
assert len(all_subIDs)==len(all_situationValues)==len(all_situValue_embeddings)

situValue_reduced_emb = [normalize_l2(emb[:256]) for emb in all_situValue_embeddings]

situationValue_data = {'ID':all_subIDs, 'text':all_situationValues, 'embedding':all_situValue_embeddings, 'embedding_reduced':situValue_reduced_emb}
with open(f'{os.getcwd()}/data/AITA/situationValueDistinct_text-embedding-3-large.pkl', 'wb') as f:
    pickle.dump(situationValue_data, f)

### Abstract Situation value embedding (aggregated)
- **situ_val**: raw generation from gpt
- **situ_val_pattern_cats**: clustered gpt-gen values
- **situ_val_patterns**: unifying concepts generated from clusters (diversifying cluster items)

In [17]:
all_subIDs, all_situationValues = [], []
for idx, row in df_values.iterrows():
    all_subIDs.append(row['subID'])
    all_situationValues.append(row['situ_val_patterns'])
    
all_situValue_embeddings = get_embeddings(all_situationValues)
assert len(all_subIDs)==len(all_situationValues)==len(all_situValue_embeddings)

situValue_reduced_emb = [normalize_l2(emb[:256]) for emb in all_situValue_embeddings]

situationValue_data = {'ID':all_subIDs, 'text':all_situationValues, 'embedding':all_situValue_embeddings, 'embedding_reduced':situValue_reduced_emb}
with open(f'{os.getcwd()}/embedding/ConceptValue_text-embedding-3-large.pkl', 'wb') as f:
    pickle.dump(situationValue_data, f)

100%|████████████████████████████████████████████████████████████████████████| 124/124 [02:15<00:00,  1.09s/it]


### Comment Value embedding

In [89]:
all_IDs, all_commValues = [], []
for idx, row in df_values.iterrows():
    if row['comm_val_patterns'] != row['comm_val_patterns'] or row['comm_val_patterns'] == '':
        continue
    all_IDs.append(row['subID']+'###'+row['author'])
    all_commValues.append(row['comm_val_patterns'])
    
all_commValue_embeddings = get_embeddings(all_commValues)
assert len(all_IDs)==len(all_commValues)==len(all_commValue_embeddings)

commValue_reduced_emb = [normalize_l2(emb[:256]) for emb in all_commValue_embeddings]

commValue_data = {'ID':all_IDs, 'text':all_commValues, 'embedding':all_commValue_embeddings, 'embedding_reduced':commValue_reduced_emb}
with open(f'{os.getcwd()}/embedding/Comm-Value_text-embedding-3-large.pkl', 'wb') as f:
    pickle.dump(commValue_data, f)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 156/156 [03:46<00:00,  1.45s/it]


### Situation's Schwartz value embedding

In [41]:
with open(f'{os.getcwd()}/data/subID_to_SchwartzValueMapping_4o.pkl', 'rb') as f:
    subID_to_schwartz = pickle.load(f)
with open(f'{os.getcwd()}/data/subID_to_SchwartzValueMapping_4o-mini.pkl', 'rb') as f:
    subID_to_schwartz_mini = pickle.load(f)

In [42]:
schwartz_values = ['self-direction', 'universalism', 'benevolence', 'conformity', 'tradition', 
                   'security', 'power', 'achievement', 'hedonism', 'stimulation']
schwartz_value_cats = ['openness to change', 'self-transcendence', 'conservation', 'self-enhancement']

**Count only the valid ones**

In [43]:
subID_to_salientSchwartz = {}
for _subID in subID_to_schwartz:
    text_list = subID_to_schwartz[_subID].split('\n')
    _curr_valid_schwartzs = {key:0 for key in schwartz_values + schwartz_value_cats}
    for elem in text_list:
        elem = elem[1:].strip().lower() if elem.startswith('-') else elem.strip().lower()
        if 'vs.' not in elem:
            continue

        for item in schwartz_values + schwartz_value_cats:
            _val_cnt = elem.count(item)
            _curr_valid_schwartzs[item] += _val_cnt

    _curr_valid_schwartzs_sorted = sorted(_curr_valid_schwartzs.items(), key=lambda x:x[1], reverse=True)
    subID_to_salientSchwartz[_subID] = [elem[0] for elem in _curr_valid_schwartzs_sorted[:3]]

In [67]:
all_subIDs, all_Schwartz = [], []
for _subID in subID_to_salientSchwartz:
    all_subIDs.append(_subID)
    all_Schwartz.append(' '.join(subID_to_salientSchwartz[_subID]))
    
all_Schwartz_embeddings = get_embeddings(all_Schwartz)
assert len(all_subIDs)==len(all_Schwartz)==len(all_Schwartz_embeddings)

Schwartz_reduced_emb = [normalize_l2(emb[:256]) for emb in all_Schwartz_embeddings]

Schwartz_data = {'ID':all_subIDs, 'text':all_Schwartz, 'embedding':all_Schwartz_embeddings, 'embedding_reduced':Schwartz_reduced_emb}
with open(f'{os.getcwd()}/embedding/situationSchwartzValue_text-embedding-3-large.pkl', 'wb') as f:
    pickle.dump(Schwartz_data, f)

100%|██████████████████████████████████████████████████████████████████████████| 13/13 [00:51<00:00,  3.94s/it]


## Get distance

In [5]:
from sklearn import metrics

def get_pairwise_dists(X, Y, emb_dim=256, thr=100000):
    all_dists = []
    for i in tqdm(range(len(X))):
        _dists = metrics.pairwise_distances(np.reshape(X[i],(1,emb_dim)),np.array(Y))[0]
        all_dists.append(_dists[:thr])
    return all_dists

### Pairwise distance by situation embedding

In [12]:
with open(f'{os.getcwd()}/embedding/situation_text-embedding-3-large.pkl', 'rb') as f:
    situation_data = pickle.load(f)

situ_dist = get_pairwise_dists(situation_data['embedding_reduced'],situation_data['embedding_reduced'])
with open(f'{os.getcwd()}/data/situation_pairwiseDists_text-embedding-3-large_reduced.pkl', 'wb') as f:
    pickle.dump({'ID':situation_data['ID'], 'dists':situ_dist}, f)

100%|███████████████████████████████████████████████████████████████████| 17432/17432 [02:38<00:00, 109.89it/s]


### Pairwise distance by Schwartz's embedding

In [13]:
with open(f'{os.getcwd()}/embedding/situationSchwartzValue_text-embedding-3-large.pkl', 'rb') as f:
    situSchwartz_data = pickle.load(f)

situSchwartz_dist = get_pairwise_dists(situSchwartz_data['embedding_reduced'],situSchwartz_data['embedding_reduced'])
with open(f'{os.getcwd()}/data/situationSchwartzValue_pairwiseDists_text-embedding-3-large_reduced.pkl', 'wb') as f:
    pickle.dump({'ID':situSchwartz_data['ID'], 'dists':situSchwartz_dist}, f)

100%|███████████████████████████████████████████████████████████████████| 17432/17432 [02:37<00:00, 110.53it/s]


### Pairwise distance by value (Agg) embedding

In [14]:
with open(f'{os.getcwd()}/embedding/situationValueAgg_text-embedding-3-large.pkl', 'rb') as f:
    situValue_data = pickle.load(f)

situVal_dist = get_pairwise_dists(situValue_data['embedding_reduced'],situValue_data['embedding_reduced'])
with open(f'{os.getcwd()}/data/situationValueAgg_pairwiseDists_text-embedding-3-large_reduced.pkl', 'wb') as f:
    pickle.dump({'ID':situValue_data['ID'], 'dists':situVal_dist}, f)

100%|███████████████████████████████████████████████████████████████████| 17432/17432 [02:36<00:00, 111.28it/s]


### Pairwise distance by value (Distinct) embedding

In [21]:
with open(f'{os.getcwd()}/embedding/ConceptValue_text-embedding-3-large.pkl', 'rb') as f:
    situValue_data = pickle.load(f)

situVal_dist = get_pairwise_dists(situValue_data['embedding_reduced'],situValue_data['embedding_reduced'])
with open(f'{os.getcwd()}/embedding/pairwise_dists/ConceptValue_text-embedding-3-large_reduced.pkl', 'wb') as f:
    pickle.dump({'ID':situValue_data['ID'], 'dists':situVal_dist}, f)

100%|███████████████████████████████████████████████████████████████████| 18669/18669 [02:57<00:00, 104.89it/s]


## Pairwise distance btw Situation and Comment Value embedding

In [6]:
with open(f'{os.getcwd()}/embedding/Value_text-embedding-3-large.pkl', 'rb') as f:
    situValue_data = pickle.load(f)

with open(f'{os.getcwd()}/embedding/Comm-Value_text-embedding-3-large.pkl', 'rb') as f:
    commValue_data = pickle.load(f)

In [7]:
situ_dist = get_pairwise_dists(situValue_data['embedding_reduced'],commValue_data['embedding_reduced'])

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 17432/17432 [07:48<00:00, 37.21it/s]


In [8]:
dist_data = {'ID':situValue_data['ID'], 'refID':commValue_data['ID'], 'dists':situ_dist}
with open(f'{os.getcwd()}/embedding/pairwise_dists/Comm-Value_text-embedding-3-large_reduced.pkl', 'wb') as f:
    pickle.dump(dist_data, f)

### Get the closest instances by situation

In [11]:
author_to_participatingSubIDs = {}
AUTHORS = list(set(df_ext.commAuthor.tolist()))
for _author in AUTHORS:
    minidf = df_ext[df_ext['commAuthor']==_author]
    author_to_participatingSubIDs[_author] = minidf.subID.tolist()

In [12]:
subID_to_allAuthors = {}
for idx, row in tqdm(df_ext.iterrows()):
    if row['subID'] in subID_to_allAuthors:
        continue
    subID_to_allAuthors[row['subID']] = df_ext[df_ext['subID']==row['subID']].commAuthor.tolist()

52960it [00:33, 1567.21it/s]


In [19]:
"""
- For each situation
    - Sort pairwise dists
    - Get all authors participating the situation
    - For each author, participating the situation
        - Highlight the sorted dists with the situations that this author participate
"""

with open(f'{os.getcwd()}/data/situation_pairwiseDists_text-embedding-3-large_reduced.pkl', 'rb') as f:
    situ_dist_data = pickle.load(f)
    
subID_to_situIndex, situIndex_to_subID = {}, {}
for idx, elem in enumerate(situ_dist_data['ID']):
    subID_to_situIndex[elem] = idx
    situIndex_to_subID[idx] = elem
    
ID_to_closest = {}
for idx, elem in tqdm(enumerate(situ_dist_data['dists'])):
    curr_subID = situIndex_to_subID[idx]
    if curr_subID not in subID_to_allAuthors:
        continue
        
    sorted_dists = sorted(enumerate(elem[0]), key=lambda x:x[1])
    
    curr_authors = subID_to_allAuthors[curr_subID]
    for _author in curr_authors:
        curr_author_subIDs = author_to_participatingSubIDs[_author]
        curr_author_situIndices = [subID_to_situIndex[item] for item in curr_author_subIDs]
        curr_closest = [item for item in sorted_dists[1:] if item[0] in curr_author_situIndices]

        ID_to_closest[curr_subID+'###'+_author] = curr_closest[:100]

with open(f"{os.getcwd()}/data/ID_to_closest_bySituation.pkl", "wb") as f:
    pickle.dump(ID_to_closest, f)

17432it [1:14:58,  3.88it/s]


In [20]:
import gc
del ID_to_closest
gc.collect()

with open(f'{os.getcwd()}/data/situationValueAgg_pairwiseDists_text-embedding-3-large_reduced.pkl', 'rb') as f:
    situ_dist_data = pickle.load(f)
    
subID_to_situIndex, situIndex_to_subID = {}, {}
for idx, elem in enumerate(situ_dist_data['ID']):
    subID_to_situIndex[elem] = idx
    situIndex_to_subID[idx] = elem

ID_to_closest = {}
for idx, elem in tqdm(enumerate(situ_dist_data['dists'])):
    curr_subID = situIndex_to_subID[idx]
    if curr_subID not in subID_to_allAuthors:
        continue
        
    sorted_dists = sorted(enumerate(elem[0]), key=lambda x:x[1])
    
    curr_authors = subID_to_allAuthors[curr_subID]
    for _author in curr_authors:
        curr_author_subIDs = author_to_participatingSubIDs[_author]
        curr_author_situIndices = [subID_to_situIndex[item] for item in curr_author_subIDs]
        curr_closest = [item for item in sorted_dists[1:] if item[0] in curr_author_situIndices]

        ID_to_closest[curr_subID+'###'+_author] = curr_closest[:100]

with open(f"{os.getcwd()}/data/ID_to_closest_bySituationValueAgg.pkl", "wb") as f:
    pickle.dump(ID_to_closest, f)

17432it [1:14:20,  3.91it/s]


In [21]:
del ID_to_closest
gc.collect()

with open(f'{os.getcwd()}/data/situationSchwartzValue_pairwiseDists_text-embedding-3-large_reduced.pkl', 'rb') as f:
    situ_dist_data = pickle.load(f)
    
subID_to_situIndex, situIndex_to_subID = {}, {}
for idx, elem in enumerate(situ_dist_data['ID']):
    subID_to_situIndex[elem] = idx
    situIndex_to_subID[idx] = elem

ID_to_closest = {}
for idx, elem in tqdm(enumerate(situ_dist_data['dists'])):
    curr_subID = situIndex_to_subID[idx]
    if curr_subID not in subID_to_allAuthors:
        continue
        
    sorted_dists = sorted(enumerate(elem[0]), key=lambda x:x[1])
    
    curr_authors = subID_to_allAuthors[curr_subID]
    for _author in curr_authors:
        curr_author_subIDs = [item for item in author_to_participatingSubIDs[_author] if item != curr_subID]
        curr_author_situIndices = [subID_to_situIndex[item] for item in curr_author_subIDs]
        curr_closest = [item for item in sorted_dists[1:] if item[0] in curr_author_situIndices]

        assert curr_subID+'###'+_author in ID_to_closest
        ID_to_closest[curr_subID+'###'+_author] = curr_closest[:100]
        
        for item in ID_to_closest[curr_subID+'###'+_author]:
            if situIndex_to_subID[item[0]] not in author_to_participatingSubIDs[_ID.split('###')[1]]:
                print(idx, elem, _author, curr_subID)
                break

with open(f"{os.getcwd()}/data/ID_to_closest_bySituationSchwartzValue.pkl", "wb") as f:
    pickle.dump(ID_to_closest, f)

17432it [1:10:23,  4.13it/s]


In [26]:

with open(f'{os.getcwd()}/embedding/pairwise_dists/ConceptValue_text-embedding-3-large_reduced.pkl', 'rb') as f:
    situ_dist_data = pickle.load(f)
    
subID_to_situIndex, situIndex_to_subID = {}, {}
for idx, elem in enumerate(situ_dist_data['ID']):
    subID_to_situIndex[elem] = idx
    situIndex_to_subID[idx] = elem

ID_to_closest = {}
for idx, elem in tqdm(enumerate(situ_dist_data['dists'])):
    curr_subID = situIndex_to_subID[idx]
    if curr_subID not in subID_to_allAuthors:
        continue
        
    sorted_dists = sorted(enumerate(elem), key=lambda x:x[1])
    
    curr_authors = subID_to_allAuthors[curr_subID]
    for _author in curr_authors:
        curr_author_subIDs = [item for item in author_to_participatingSubIDs[_author] if item != curr_subID]
        curr_author_situIndices = [subID_to_situIndex[item] for item in curr_author_subIDs]
        curr_closest = [item for item in sorted_dists[1:] if item[0] in curr_author_situIndices]

        ID_to_closest[curr_subID+'###'+_author] = curr_closest[:100]

with open(f"{os.getcwd()}/data/ID_to_closest_byConceptValue.pkl", "wb") as f:
    pickle.dump(ID_to_closest, f)

18669it [1:36:20,  3.23it/s]


In [13]:
"""
- For each situation
    - Sort pairwise dists
    - Get all authors participating the situation
    - For each author, participating the situation
        - Highlight the sorted dists with the situations that this author participate
"""

# with open(f'{os.getcwd()}/embedding/pairwise_dists/Situation-Comment-Value_text-embedding-3-large_reduced.pkl', 'rb') as f:
#     dist_data = pickle.load(f)
    
subID_to_situIndex, situIndex_to_subID = {}, {}
for idx, elem in enumerate(dist_data['ID']):
    situIndex_to_subID[idx] = elem

ID_to_situIndex, situIndex_to_ID = {}, {}
for idx, elem in enumerate(dist_data['refID']):
    ID_to_situIndex[elem] = idx
    
ID_to_closest = {}
for idx, elem in tqdm(enumerate(dist_data['dists'])):
    curr_subID = situIndex_to_subID[idx]
    if curr_subID not in subID_to_allAuthors:
        continue
        
    sorted_dists = sorted(enumerate(elem), key=lambda x:x[1])
    
    curr_authors = subID_to_allAuthors[curr_subID]
    for _author in curr_authors:
        curr_author_subIDs = author_to_participatingSubIDs[_author]
        curr_author_IDs = [item+'###'+_author for item in curr_author_subIDs if item != curr_subID]
        curr_author_situIndices = [ID_to_situIndex[item] for item in curr_author_IDs if item in ID_to_situIndex]
        curr_closest = [item for item in sorted_dists if item[0] in curr_author_situIndices]

        ID_to_closest[curr_subID+'###'+_author] = curr_closest[:100]

with open(f"{os.getcwd()}/data/ID_to_closest_byComm-Value.pkl", "wb") as f:
    pickle.dump(ID_to_closest, f)

17432it [4:09:25,  1.16it/s]


### Get the closest instance for values, considering situation distance

**Schwartz values**

In [ ]:
with open(f'{os.getcwd()}/embedding/pairwise_dists/situation_text-embedding-3-large_reduced.pkl', 'rb') as f:
    situation_data_dists = pickle.load(f)

with open(f"{os.getcwd()}/data/ID_to_closest_bySituationSchwartzValue.pkl", "rb") as f:
    ID_to_closestSchwartz = pickle.load(f)

In [66]:
subID_to_situIndex, situIndex_to_subID = {}, {}
for idx, elem in enumerate(situation_data_dists['ID']):
    subID_to_situIndex[elem] = idx
    situIndex_to_subID[idx] = elem

In [70]:
ID_to_closestSchwartz_bySituation = {}
for _ID in tqdm(ID_to_closestSchwartz):
    situIndex = subID_to_situIndex[_ID.split('###')[0]]
    _withSituDist = [(elem[0],elem[1],situation_data_dists['dists'][situIndex][0][elem[0]]) for elem in ID_to_closestSchwartz[_ID]]
    sorted_closest_bySchwartz = sorted(_withSituDist, key=lambda x:(x[1], x[2]))
    ID_to_closestSchwartz_bySituation[_ID] = sorted_closest_bySchwartz

100%|██████████████████████████████████████████████████████████████████| 52812/52812 [00:09<00:00, 5427.68it/s]


In [154]:
cnt1, cnt2 = 0, 0
lengths = []
for _ID in tqdm(ID_to_closestSchwartz_bySituation):
    lengths.append(len(ID_to_closestSchwartz_bySituation[_ID]))
    for item in ID_to_closestSchwartz_bySituation[_ID]:
        if situIndex_to_subID[item[0]] not in author_to_participatingSubIDs[_ID.split('###')[1]]:
            cnt1 += 1
        else:
            cnt2 += 1
print(cnt1, cnt2)
len_cnt = Counter()
len_cnt.update(lengths)
len_cnt = sorted(len_cnt.items(), key=lambda x:x[1])
len_cnt[:10]

100%|██████████████████████████████████████████████████████████████████| 52812/52812 [00:20<00:00, 2634.74it/s]

0 5280970


[(98, 13), (99, 204), (100, 52595)]

In [155]:
with open(f"{os.getcwd()}/data/ID_to_closest_bySchwartzValue_bySituation.pkl", "wb") as f:
    pickle.dump(ID_to_closestSchwartz_bySituation, f)

**Agg Values**

In [166]:
with open(f'{os.getcwd()}/embedding/pairwise_dists/AggValue_text-embedding-3-large_reduced.pkl', 'rb') as f:
    situ_dist_data = pickle.load(f)
    
subID_to_situIndex, situIndex_to_subID = {}, {}
for idx, elem in enumerate(situ_dist_data['ID']):
    subID_to_situIndex[elem] = idx
    situIndex_to_subID[idx] = elem

In [156]:
with open(f"{os.getcwd()}/data/ID_to_closest_byAggValue.pkl", "rb") as f:
    ID_to_closest = pickle.load(f)

In [167]:
ID_to_closest_bySituation = {}
for _ID in tqdm(ID_to_closest):
    situIndex = subID_to_situIndex[_ID.split('###')[0]]
    _withSituDist = [(elem[0],elem[1],situation_data_dists['dists'][situIndex][0][elem[0]]) for elem in ID_to_closest[_ID]]
    sorted_closest = sorted(_withSituDist, key=lambda x:(x[1], x[2]))
    ID_to_closest_bySituation[_ID] = sorted_closest

100%|██████████████████████████████████████████████████████████████████| 52812/52812 [00:16<00:00, 3224.81it/s]


In [169]:
cnt1, cnt2 = 0, 0
for _ID in tqdm(ID_to_closest_bySituation):
    for item in ID_to_closest_bySituation[_ID]:
        if situIndex_to_subID[item[0]] not in author_to_participatingSubIDs[_ID.split('###')[1]]:
            cnt1 += 1
        else:
            cnt2 += 1
print(cnt1, cnt2)

100%|██████████████████████████████████████████████████████████████████| 52812/52812 [00:20<00:00, 2560.51it/s]

0 5280302


In [170]:
with open(f"{os.getcwd()}/data/ID_to_closest_byAggValue_bySituation.pkl", "wb") as f:
    pickle.dump(ID_to_closest_bySituation, f)

**ConceptValue**

In [27]:
with open(f'{os.getcwd()}/embedding/pairwise_dists/ConceptValue_text-embedding-3-large_reduced.pkl', 'rb') as f:
    situ_dist_data = pickle.load(f)
    
subID_to_situIndex, situIndex_to_subID = {}, {}
for idx, elem in enumerate(situ_dist_data['ID']):
    subID_to_situIndex[elem] = idx
    situIndex_to_subID[idx] = elem

In [28]:
with open(f"{os.getcwd()}/data/ID_to_closest_byConceptValue.pkl", "rb") as f:
    ID_to_closest = pickle.load(f)

In [31]:
ID_to_closest_bySituation = {}
for _ID in tqdm(ID_to_closest):
    situIndex = subID_to_situIndex[_ID.split('###')[0]]
    _withSituDist = [(elem[0],elem[1],situ_dist_data['dists'][situIndex][elem[0]]) for elem in ID_to_closest[_ID]]
    sorted_closest = sorted(_withSituDist, key=lambda x:(x[1], x[2]))
    ID_to_closest_bySituation[_ID] = sorted_closest

100%|██████████████████████████████████████████████████████████████████| 52812/52812 [00:27<00:00, 1933.33it/s]


In [32]:
cnt1, cnt2 = 0, 0
for _ID in tqdm(ID_to_closest_bySituation):
    for item in ID_to_closest_bySituation[_ID]:
        if situIndex_to_subID[item[0]] not in author_to_participatingSubIDs[_ID.split('###')[1]]:
            cnt1 += 1
        else:
            cnt2 += 1
print(cnt1, cnt2)

100%|██████████████████████████████████████████████████████████████████| 52812/52812 [00:22<00:00, 2324.83it/s]

0 5280302


In [33]:
with open(f"{os.getcwd()}/data/ID_to_closest_byConceptValue_bySituation.pkl", "wb") as f:
    pickle.dump(ID_to_closest_bySituation, f)

# Create data instance for prompting

## Additional value annotation data

In [64]:
df_values_addtl = pd.read_csv(f'{os.getcwd()}/gpt_inference/directComms_withValuesRefined.tsv', sep='\t')

In [244]:
def process_json_outputs(item):
    item = item.replace("'Conflicting Values'",'"Conflicting Values"').replace("'Winning Values'",'"Winning Values"')
    item = item.replace("\'Conflicting Values\'",'"Conflicting Values"').replace("\'Winning Values\'",'"Winning Values"')
    item = item.replace("],\n}","]\n}")
    item = item.replace(",\n    ]","\n    ]")
    item = item.replace("    '",'    "').replace("',\n",'",\n').replace("'\n",'"\n')
    
    if item.startswith("```json"):
        item = item.replace("```json","")
        item = item.split("```")[0]
    elif item.startswith("```python"):
        item = item.replace("```python","")
        item = item.split("```")[0]
        
    if "//" in item:
        item1 = item.split("//")[0]
        item2 = item.split("//")[1].split("\n")[0]
        item = item.replace(item2,'').replace("//","")
    if "}" not in item:
        item += "    ]\n}"
        item = item.replace(",\n    ]","\n    ]")
        
    return item.strip()

In [97]:
author_to_subID_to_value = {}
for idx, row in df_values_addtl.iterrows():
    if row['author'] not in author_to_subID_to_value:
        author_to_subID_to_value[row['author']] = {}
    author_to_subID_to_value[row['author']][row['subID']] = [row['situ_val'], json.loads(process_json_outputs(row['comm_val']))]

In [98]:
with open(f'{os.getcwd()}/../AbstractiveSubjectivity/data/AITA/author_to_subID_to_Values_withCommValueTradeoff.pkl', 'rb') as f:
    author_to_subID_to_Values_orig = pickle.load(f)

In [99]:
for _author in author_to_subID_to_value:
    if _author in author_to_subID_to_Values_orig:
        author_to_subID_to_value[_author] = author_to_subID_to_value[_author] | author_to_subID_to_Values_orig[_author]

In [100]:
with open(f'{os.getcwd()}/data/author_to_subID_to_Values_withCommValueTradeoff.pkl', 'wb') as f:
    pickle.dump(author_to_subID_to_value, f)

For newly added comments,
1. Check if the annotated comment values match the situation values. If so, assign their patterns
2. If not, compute the distance  
    2-1. Get embeddings for all situ_vals  
    2-2. Get embeddings for comm_vals that don't align  
    2-3. Get the closest situ_val for each comm_val  
    2-4. Assign the same situ_val_catetories and patterns  

In [8]:
import openai
from openai import OpenAI
openai.api_key = os.getenv("OPENAI_API_KEY")
client = OpenAI()

import tiktoken
encoding = tiktoken.encoding_for_model("gpt-4o")

def slice_text_list(text_list, thr=4000):
    cnt = 0
    final_slice, sub_slice = [], []
    for text in text_list:
        encoded = encoding.encode(text)
        if cnt + len(encoded) < thr and len(sub_slice) < 500:
            sub_slice.append(text)
            cnt += len(encoded)
        else:
            final_slice.append(sub_slice)
            sub_slice = [text]
            cnt = len(encoded)
    final_slice.append(sub_slice)
    return final_slice

def normalize_l2(x):
    x = np.array(x)
    if x.ndim == 1:
        norm = np.linalg.norm(x)
        if norm == 0:
            return x
        return x / norm
    else:
        norm = np.linalg.norm(x, 2, axis=1, keepdims=True)
        return np.where(norm == 0, x, x / norm)

In [13]:
df['id'] = df['subID']+'###'+df['commAuthor']
df_values['id'] = df_values['subID']+'###'+df_values['author']

df_values_addtl = df_values[~df_values['id'].isin(df['id'].tolist())]
len(df), len(df_values), len(df_values_addtl)

(18669, 53280, 34611)

In [81]:
df_orig = df_values[df_values['id'].isin(df['id'].tolist())]
len(df), len(df_values), len(df_values_addtl), len(df_orig)

(18669, 53280, 34611, 18669)

In [15]:
df_values_addtl.head(2)

,subID,title,selftext,author,comment,judgment,situ_val,comm_val,situ_val_patterns,situ_val_pattern_cats,comm_val_patterns,comm_val_pattern_cats,id
18669,h0lfjt,not wanting to share my wedding day with my br...,My (27F) fiancée (28M) and I have been dating ...,DocSternau,"Ah, allways with the wedding drama / competiti...",1,"['Individuality vs. Family Unity', 'Personal D...","```json\n{\n ""Conflicting Values"": [\n ...",Authenticity and Individuality vs. Family Unit...,Themes of Identity Exploration vs. Desire for ...,n > a\nn > e,e > n\ne > s,h0lfjt###DocSternau
18670,h0ow13,I refuse to take responsibility for my daughte...,My(48) daughter(22) has had a laundry list of ...,Dachshundmom5,"your own post shows verbal abuse, manipulation...",0,"['Responsibility vs. Blame', ""Parental expecta...","```json\n{\n ""Conflicting Values"": [\n ...",Commitment and Responsibility vs. Blame Assign...,Commitment and Agreements Framework vs. Emotio...,C > C\nd > m\nu > m,P > P\nr > m\ni > m,h0ow13###Dachshundmom5


In [82]:
df_orig.head(2)

,subID,title,selftext,author,comment,judgment,situ_val,comm_val,situ_val_patterns,situ_val_pattern_cats,comm_val_patterns,comm_val_pattern_cats,id
0,k8zq74,not doing my stepchildren's hair?,"Hello, I'm 30F married to my partner 51M and h...",NoApollonia,The children are both old enough to choose if ...,1,"[""BM's value of appearance and expectation for...","```json\n{\n ""Conflicting Values"": [\n ...",Social Appearance and Judgment vs. Family Dyna...,Societal Norms and Individual Impact vs. Perso...,Self-Expression and Comfort > Parenting Author...,Themes of Identity Exploration > Parenting Dec...,k8zq74###NoApollonia
1,k93tjn,insulting my wife in front of our son.,Growing up I had faced a few key issues that I...,NoApollonia,Don't disrespect your wife in front of your so...,0,['Personal Achievement vs. Academic Excellence...,"```json\n{\n ""Conflicting Values"": [\n ...",Personal Career Satisfaction vs. Academic Pres...,Career Development and Satisfaction Themes vs....,Academic Pressure and Expectations > Personal ...,Educational Themes and Responsibilities > Care...,k93tjn###NoApollonia


In [63]:
exist_situ_vals, exist_situ_val_IDs = [], []

for idx, row in df_values_addtl.iterrows():
    situ_items = [item for item in ast.literal_eval(row['situ_val']) if ('vs.' in item and item.split('vs.')[-1].strip() != '')]
    for i, item in enumerate(situ_items):
        item1 = item.split('vs.')[0].strip().lower()
        item2 = item.split('vs.')[-1].strip().lower()
        if item1 == '' or item2 == '':
            continue
        
        exist_situ_vals.append(item1)
        exist_situ_val_IDs.append(row['subID']+'@'+str(i)+'_1')
        exist_situ_vals.append(item2)
        exist_situ_val_IDs.append(row['subID']+'@'+str(i)+'_2')

val_to_ID = {_val:_ID for _val,_ID in zip(exist_situ_vals, exist_situ_val_IDs)}

exist_situ_vals_unique = list(set(exist_situ_vals))
all_values_sliced = slice_text_list(exist_situ_vals_unique)

all_value_embeddings = []
for values in tqdm(all_values_sliced):
    prompt_texts = [text.replace('\n',' ') for text in values]
    
    response = client.embeddings.create(input=prompt_texts,model="text-embedding-3-small")
    all_value_embeddings += [item.embedding for item in response.data]

dim_size=256
value_embeddings_reduced = [normalize_l2(emb[:dim_size]) for emb in all_value_embeddings]

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 63/63 [01:45<00:00,  1.68s/it]


In [20]:
author_to_subID_to_Values['DocSternau']['h0lfjt'][1]['Winning Values']

['Personal Dream > Financial Constraints',
 'Self-Expression > Pressure to Accommodate']

In [65]:
addtl_comm_vals, addtl_comm_IDs = [], []

for idx, row in df_values_addtl.iterrows():
    situ_items = [item for item in ast.literal_eval(row['situ_val']) if ('vs.' in item and item.split('vs.')[-1].strip() != '')]
    situ_item_distict = []
    for item in situ_items:
        item1 = item.split('vs.')[0].strip().lower()
        item2 = item.split('vs.')[-1].strip().lower()
        if item1 == '' or item2 == '':
            continue
        situ_item_distict.append(item1)
        situ_item_distict.append(item2)
        
    comm_items = author_to_subID_to_Values[row['author']][row['subID']][1]['Winning Values']
    for i, item in enumerate(comm_items):
        divider = ">" if ">" in item else "<"
        item1 = item.split(divider)[0].strip().lower()
        item2 = item.split(divider)[-1].strip().lower()

        if item1 == '' or item2 == '':
            continue
        
        if item1 not in situ_item_distict:
            addtl_comm_vals.append(item1)
            addtl_comm_IDs.append(row['subID']+'###'+row['author']+'@'+str(i)+'_1')
        if item2 not in situ_item_distict:
            addtl_comm_vals.append(item2)
            addtl_comm_IDs.append(row['subID']+'###'+row['author']+'@'+str(i)+'_2')

commVal_to_ID = {_comm:_ID for _comm,_ID in zip(addtl_comm_vals, addtl_comm_IDs)}
addtl_comm_vals_unique = list(set(addtl_comm_vals))

all_comm_sliced = slice_text_list(addtl_comm_vals_unique)

all_comm_embeddings = []
for comms in tqdm(all_comm_sliced):
    prompt_texts = [text.replace('\n',' ') for text in comms]
    
    response = client.embeddings.create(input=prompt_texts,model="text-embedding-3-small")
    all_comm_embeddings += [item.embedding for item in response.data]

dim_size=256
comm_embeddings_reduced = [normalize_l2(emb[:dim_size]) for emb in all_comm_embeddings]

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:02<00:00,  1.12s/it]


In [66]:
commVal_to_situVal = {}
for idx, _emb in tqdm(enumerate(comm_embeddings_reduced)):
    _dists = metrics.pairwise_distances(np.reshape(_emb,(1,dim_size)),
                                        np.array(value_embeddings_reduced))

    _dists_sorted = sorted(enumerate(_dists[0]), key=lambda x:x[1], reverse=False)

    commVal_to_situVal[addtl_comm_vals_unique[idx]] = exist_situ_vals_unique[_dists_sorted[0][0]]

614it [00:17, 35.22it/s]


In [67]:
subID_to_patterns, subID_to_cats = {}, {}
for idx, row in df_values.iterrows():
    subID_to_patterns[row['subID']] = row['situ_val_patterns']
    subID_to_cats[row['subID']] = row['situ_val_pattern_cats']

In [36]:
subID_to_patterns['h0lfjt']

'Authenticity and Individuality vs. Family Unity Seeking\nPursuit of Personal Dreams vs. Economic Pressures and Constraints\nBehavioral Consequences of Entitlement vs. Meritocracy and Fairness\nCelebration Significance and Value vs. Support Systems and Community\nSelf-Expression and Comfort vs. Dependency and Agency'

In [69]:
Val_to_patterns, pattern_to_Cats = {}, {}
for idx, row in df_values_addtl.iterrows():
    situ_items = [item for item in ast.literal_eval(row['situ_val']) if ('vs.' in item and item.split('vs.')[-1].strip() != '')]
    patterns = subID_to_patterns[row['subID']].split('\n')
    cats = subID_to_cats[row['subID']].split('\n')

    # if len(situ_items) != len(patterns):
    #     continue
    # if len(situ_items) > len(patterns):
    #     situ_items = situ_items[:len(patterns)]
    # elif len(patterns) > len(situ_items):
    #     patterns = patterns[:len(situ_items)]
    #     cats = cats[:len(situ_items)]

    assert len(situ_items)==len(patterns)==len(cats)

    for _situ, _pattern, _cat in zip(situ_items, patterns, cats):
        _val1 = _situ.split('vs.')[0].strip().lower()
        _val2 = _situ.split('vs.')[-1].strip().lower()
        _p1 = _pattern.split('vs.')[0].strip()
        _p2 = _pattern.split('vs.')[-1].strip()
        _c1 = _cat.split('vs.')[0].strip()
        _c2 = _cat.split('vs.')[-1].strip()
        # if '' in [_val1, _val2, _p1, _p2, _c1, _c2]:
        #     print('dd')
        Val_to_patterns[_val1] = _p1
        Val_to_patterns[_val2] = _p2

        pattern_to_Cats[_p1]=_c1
        pattern_to_Cats[_p2]=_c2

In [72]:
author_to_subID_to_CommValue_assignment['DocSternau']['h0lfjt'][0]

'Pursuit of Personal Dreams > Economic Pressures and Constraints\nSelf-Expression and Comfort > Dependency and Agency'

In [70]:
author_to_subID_to_CommValue_assignment = {}

for idx, row in df_values_addtl.iterrows():
    if row['author'] not in author_to_subID_to_CommValue_assignment:
        author_to_subID_to_CommValue_assignment[row['author']] = {}

    situ_items = [item for item in ast.literal_eval(row['situ_val']) if ('vs.' in item and item.split('vs.')[-1].strip() != '')]
    situ_item_distict = []
    for item in situ_items:
        item1 = item.split('vs.')[0].strip().lower()
        item2 = item.split('vs.')[-1].strip().lower()
        if item1 == '' or item2 == '':
            continue
        situ_item_distict.append(item1)
        situ_item_distict.append(item2)

    comm_items = author_to_subID_to_Values[row['author']][row['subID']][1]['Winning Values']

    pattern_string, cat_string = "", ""
    for i, item in enumerate(comm_items):
        divider = ">" if ">" in item else "<"
        item1 = item.split(divider)[0].strip().lower()
        item2 = item.split(divider)[-1].strip().lower()
        if item1 == '' or item2 == '':
            continue
        if item1 not in situ_item_distict:
            val1 = commVal_to_situVal[item1].lower()
        else:
            val1 = item1
        if item2 not in situ_item_distict:
            val2 = commVal_to_situVal[item2].lower()
        else:
            val2 = item2
        if divider == "<":
            pattern_string += Val_to_patterns[val2] + f" > " + Val_to_patterns[val1] + "\n"
            cat_string += pattern_to_Cats[Val_to_patterns[val2]] + f" > " + pattern_to_Cats[Val_to_patterns[val1]] + "\n"
        else:
            pattern_string += Val_to_patterns[val1] + f" > " + Val_to_patterns[val2] + "\n"
            cat_string += pattern_to_Cats[Val_to_patterns[val1]] + f" > " + pattern_to_Cats[Val_to_patterns[val2]] + "\n"
            
    author_to_subID_to_CommValue_assignment[row['author']][row['subID']] = [pattern_string[:-1], cat_string[:-1]]

In [73]:
situ_val_patterns_addtl, situ_val_pattern_cats_addtl = [],[]
comm_val_patterns_addtl, comm_val_pattern_cats_addtl = [],[]
for idx, row in df_values_addtl.iterrows():
    situ_val_patterns_addtl.append(subID_to_patterns[row['subID']])
    situ_val_pattern_cats_addtl.append(subID_to_cats[row['subID']])
    comm_val_patterns_addtl.append(author_to_subID_to_CommValue_assignment[row['author']][row['subID']][0])
    comm_val_pattern_cats_addtl.append(author_to_subID_to_CommValue_assignment[row['author']][row['subID']][1])

In [74]:
df_values_addtl['situ_val_patterns']=situ_val_patterns_addtl
df_values_addtl['situ_val_pattern_cats']=situ_val_pattern_cats_addtl
df_values_addtl['comm_val_patterns']=comm_val_patterns_addtl
df_values_addtl['comm_val_pattern_cats']=comm_val_pattern_cats_addtl

/var/folders/xg/3nchppc93qd7twxj7g3hstfh0000gn/T/ipykernel_70734/1267250259.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_values_addtl['situ_val_patterns']=situ_val_patterns_addtl
/var/folders/xg/3nchppc93qd7twxj7g3hstfh0000gn/T/ipykernel_70734/1267250259.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_values_addtl['situ_val_pattern_cats']=situ_val_pattern_cats_addtl
/var/folders/xg/3nchppc93qd7twxj7g3hstfh0000gn/T/ipykernel_70734/1267250259.py:3: SettingWithCopyWarning: 
A value is trying t

In [84]:
df_values_all

,subID,title,selftext,author,comment,judgment,situ_val,comm_val,situ_val_patterns,situ_val_pattern_cats,comm_val_patterns,comm_val_pattern_cats,id
0,k8zq74,not doing my stepchildren's hair?,"Hello, I'm 30F married to my partner 51M and h...",NoApollonia,The children are both old enough to choose if ...,1,"[""BM's value of appearance and expectation for...","```json\n{\n ""Conflicting Values"": [\n ...",Social Appearance and Judgment vs. Family Dyna...,Societal Norms and Individual Impact vs. Perso...,Self-Expression and Comfort > Parenting Author...,Themes of Identity Exploration > Parenting Dec...,k8zq74###NoApollonia
1,k93tjn,insulting my wife in front of our son.,Growing up I had faced a few key issues that I...,NoApollonia,Don't disrespect your wife in front of your so...,0,['Personal Achievement vs. Academic Excellence...,"```json\n{\n ""Conflicting Values"": [\n ...",Personal Career Satisfaction vs. Academic Pres...,Career Development and Satisfaction Themes vs....,Academic Pressure and Expectations > Personal ...,Educational Themes and Responsibilities > Care...,k93tjn###NoApollonia
2,k96h8e,telling my daughter to suck it up?,I (43M) and my wife (45F) have three kids: 12F...,NoApollonia,While I agree the two sisters should share a r...,0,"['Parental authority vs. child autonomy', 'Fin...","```json\n{\n ""Conflicting Values"": [\n ...",Parenting Authority Dynamics vs. Daughter's Au...,Parenting Decision-Making and Interactions vs....,Equity in Resource Distribution > Balancing In...,Fairness and Equity Themes > Relationship Dyna...,k96h8e###NoApollonia
3,k9o2vg,refusing to help my husband financially,So firstly English isn't my first language so ...,WebbieVanderquack,"You don't have to give him 1K, especially if y...",0,"['Financial dependence vs. independence', 'Sup...","```json\n{\n ""Conflicting Values"": [\n ...",Economic Pressures and Constraints vs. Indepen...,Financial Awareness and Responsibility Analysi...,Types of Support > Future Security and Stabili...,Exploring Support Dynamics in Text > Future-Or...,k9o2vg###WebbieVanderquack
4,k9om9o,telling my friend to take off her makeup?,I'm new to this but just a few minutes ago me ...,WebbieVanderquack,both for not using punctuation and for telling...,0,['Concern for health and well-being vs. Respec...,"```json\n{\n ""Conflicting Values"": [\n ...",Health and Safety Concerns vs. Respect for Age...,Health-Safety Awareness and Education vs. Pers...,disrecollectful and insensitivity > Friendship...,Emotional and Social Sensitivity Issues > Expl...,k9om9o###WebbieVanderquack
...,...,...,...,...,...,...,...,...,...,...,...,...,...
53275,c0dsnr,telling my poor friend that if she stopped spe...,I’ve been friends with Melanie for 3 years (we...,annshine,"Yeah I'm with you, and wonder if its because i...",0,['Responsibility and financial literacy vs. en...,"{\n 'Conflicting Values': [\n ""Pragm...",Budgeting and Financial Management vs. Emotion...,Financial Awareness and Responsibility Analysi...,Budgeting and Financial Management > Emotional...,Financial Awareness and Responsibility Analysi...,c0dsnr###annshine
53276,c0i3oz,Introducing my GF as a Model?,So I have been dating this Girl for awhile now...,annshine,because why did you need to tell strangers how...,0,['Personal identity vs. professional identity'...,"{\n 'Conflicting Values': [\n ""Perso...",Self-Identity and Acceptance vs. Professional ...,Themes of Identity Exploration vs. Expectation...,Self-Identity and Acceptance > Professional St...,Themes of Identity Exploration > Expectations ...,c0i3oz###annshine
53277,c0i3oz,Introducing my GF as a Model?,So I have been dating this Girl for awhile now...,ZenDendou,&amp;#x200B; Pay attention next time. If she d...,0,['Personal identity vs. professional identity'...,"```json\n{\n ""Conflicting Values"": [\n ...",Self-Identity and Acceptance vs. Professional ...,Themes of Identity Exploration vs. Expectation...,Individual Attachments and Choices > Social Pe...,Interpersonal

In [83]:
df_values_all = pd.concat([df_orig, df_values_addtl], ignore_index=True)

In [85]:
df_values_all.to_csv(f'{os.getcwd()}/data/data_active_authorExt_withAbstValues.tsv', sep='\t', index=False)

### Do Schwartz's value for comments too

In [242]:
PROMPT = """
You will be given a social situation and a Redditor's comment.
Your job is to find the top two most salient Schwartz's Basic Human Values that are observed in the comment.
Follow the format below without generating any other explanation.
Input Format:
[Situation] situation_description
[Comment] redditor_comment
Output Format:
Schwartz_value_of_A, Schwartz_value_of_B
"""
author_to_subID_to_schwartz = {}
for idx, row in tqdm(df_ext.iterrows()):
    if row['commAuthor'] not in author_to_subID_to_schwartz:
        author_to_subID_to_schwartz[row['commAuthor']] = {}
    message_list = [
        {"role": "user", "content": PROMPT},
        {"role": "user", "content": f"[Situation] {row['title']} {row['selftext']}\n[Comment]\n{row['comment']}"},
    ]
    
    response = completion_with_backoff(model='gpt-4o', messages=message_list)
    author_to_subID_to_schwartz[row['commAuthor']][row['subID']] = response.choices[0].message.content
    
with open(f'{os.getcwd()}/data/author_to_subID_to_schwartz_4o.pkl', 'wb') as f:
    pickle.dump(author_to_subID_to_schwartz, f)

52960it [9:49:07,  1.50it/s]


# Create data for prompting

In [17]:
with open(f'{os.getcwd()}/data/author_to_subID_to_schwartz_4o.pkl', 'rb') as f:
    author_to_subID_to_schwartz= pickle.load(f)
    # pickle.dump(author_to_subID_to_schwartz, f)

In [183]:
df_values_all = pd.read_csv(f'{os.getcwd()}/data/data_active_authorExt_withAbstValues.tsv', sep='\t')

In [184]:
df_values_all.head(2)

,subID,title,selftext,author,comment,judgment,situ_val,comm_val,situ_val_patterns,situ_val_pattern_cats,comm_val_patterns,comm_val_pattern_cats
0,k8zq74,not doing my stepchildren's hair?,"Hello, I'm 30F married to my partner 51M and h...",NoApollonia,The children are both old enough to choose if ...,1,"[""BM's value of appearance and expectation for...","```json\n{\n ""Conflicting Values"": [\n ...",Social Appearance and Judgment vs. Family Dyna...,Societal Norms and Individual Impact vs. Perso...,Self-Expression and Comfort > Parenting Author...,Themes of Identity Exploration > Parenting Dec...
1,k93tjn,insulting my wife in front of our son.,Growing up I had faced a few key issues that I...,NoApollonia,Don't disrespect your wife in front of your so...,0,['Personal Achievement vs. Academic Excellence...,"```json\n{\n ""Conflicting Values"": [\n ...",Personal Career Satisfaction vs. Academic Pres...,Career Development and Satisfaction Themes vs....,Academic Pressure and Expectations > Personal ...,Educational Themes and Responsibilities > Care...


In [185]:
subID_to_values = {}
for idx, row in df_values_all.iterrows():
    subID_to_values[row['subID']] = "\n".join(ast.literal_eval(row['situ_val']))

In [16]:
subID_to_Abstvalues, subID_to_Abstvalues_Clustered = {}, {}
author_to_subID_to_Abstvalues, author_to_subID_to_Abstvalues_Clustered = {}, {}
author_to_subID_to_Value = {}
for idx, row in df_values.iterrows():
    if row['author'] not in author_to_subID_to_Value:
        author_to_subID_to_Value[row['author']] = {}
    if row['author'] not in author_to_subID_to_Abstvalues:
        author_to_subID_to_Abstvalues[row['author']] = {}
    if row['author'] not in author_to_subID_to_Abstvalues_Clustered:
        author_to_subID_to_Abstvalues_Clustered[row['author']] = {}
        
    # subID_to_Abstvalues[row['subID']] = row['situ_val_patterns']
    # subID_to_Abstvalues_Clustered[row['subID']] = row['situ_val_pattern_cats']
    author_to_subID_to_Value[row['author']][row['subID']] = "\n".join(author_to_subID_to_Values[row['author']][row['subID']][1]['Winning Values'])
    author_to_subID_to_Abstvalues[row['author']][row['subID']] = row['comm_val_patterns']
    author_to_subID_to_Abstvalues_Clustered[row['author']][row['subID']] = row['comm_val_pattern_cats']

In [18]:
author_to_subID_to_inst = {}
for idx, row in df_ext.iterrows():
    if row['commAuthor'] not in author_to_subID_to_inst:
        author_to_subID_to_inst[row['commAuthor']] = {}
    author_to_subID_to_inst[row['commAuthor']][row['subID']] = {'title':row['title'], 
                                                                'selftext':row['selftext'], 
                                                                'comment':row['comment'], 
                                                                'schwartz':author_to_subID_to_schwartz[row['commAuthor']][row['subID']],
                                                                'value':author_to_subID_to_Value[row['commAuthor']][row['subID']],
                                                                'abst_value':author_to_subID_to_Abstvalues[row['commAuthor']][row['subID']],
                                                                'abst_value_clustered':author_to_subID_to_Abstvalues_Clustered[row['commAuthor']][row['subID']],
                                                                'judgment':row['judgment']}

In [19]:
with open(f"{os.getcwd()}/data/author_to_subID_to_inst.pkl", "wb") as f:
    pickle.dump(author_to_subID_to_inst, f)

# Check some real data

In [250]:
df_ext.head()

,Unnamed: 0,subID,title,selftext,commAuthor,comment,judgment
0,0,h0lfjt,not wanting to share my wedding day with my br...,My (27F) fiancée (28M) and I have been dating ...,DocSternau,"Ah, allways with the wedding drama / competiti...",1
1,1,h0ow13,I refuse to take responsibility for my daughte...,My(48) daughter(22) has had a laundry list of ...,Dachshundmom5,"your own post shows verbal abuse, manipulation...",0
2,2,h0ow13,I refuse to take responsibility for my daughte...,My(48) daughter(22) has had a laundry list of ...,Lunavixen15,you don't deserve juice and a cookie for barel...,0
3,3,h0tjjz,not giving one of my friends special treatment?,"My friend, Katie (fake name) texted me today w...",teresajs,"She's clingy, demanding, and high maintenance....",1
4,4,h0eavk,"taking $15,000 out of my daughters college fun...","I have an 8 year old daughter, and to make col...",iluvcats17,You should never touch the fund for anything o...,0


In [267]:
judgment_to_word = {1: 'Acceptable', 0: 'Unacceptable'}
def process_(retrieval_strategy, test_ID):
    with open(f"{os.getcwd()}/data/ID_to_closest_by{retrieval_strategy}.pkl", "rb") as f:
        ID_to_closest = pickle.load(f)
    with open(f'{os.getcwd()}/embedding/{retrieval_strategy}_text-embedding-3-large.pkl', 'rb') as f:
        situ_dist_data = pickle.load(f)
        
    subID_to_situIndex, situIndex_to_subID = {}, {}
    for idx, elem in enumerate(situ_dist_data['ID']):
        subID_to_situIndex[elem] = idx
        situIndex_to_subID[idx] = elem
    del situ_dist_data

    _closest_subIDs = [situIndex_to_subID[elem[0]] for elem in ID_to_closest[test_ID]]
    _author = test_ID.split('###')[-1]

    for i, _subID in enumerate(_closest_subIDs[:5]):
        print(f"{i+1}-th example")
        print(author_to_subID_to_inst[_author][_subID]['title'])
        print(author_to_subID_to_inst[_author][_subID]['comment'])
        print(author_to_subID_to_inst[_author][_subID]['schwartz'])
        print(author_to_subID_to_inst[_author][_subID]['value'])
        print(author_to_subID_to_inst[_author][_subID]['abst_value'])
        print(author_to_subID_to_inst[_author][_subID]['abst_value_clustered'])
        print(judgment_to_word[author_to_subID_to_inst[_author][_subID]['judgment']])
        print()
        

In [268]:

process_("Situation", "h0eavk###iluvcats17")

1-th example
buying my son a nicer car then I bought by stepdaughter because my ex wife split the cost with me, while her father contributed nothing
since your wife is upset at you instead of being grateful that you got her daughter a car. Not understanding why she can’t work more than part time though if she has skills to work part time? Perhaps she can work more if she wants her daughter to have nicer things. And perhaps your stepdaughter could work too and you could agree to match whatever she can save for a car up to a specific dollar amount.
Security, Achievement
Financial Capability > Parental Responsibility
Recognition of Contribution > Sense of Entitlement
Recognition of Efforts and Contributions > Perceived Entitlement Dynamics
Personal Financial Responsibility > Parental Responsibility and Care
Recognition and Appreciation Patterns > Entitlement and Privilege Dynamics Analysis
Financial Awareness and Responsibility Analysis > Daughter's Development and Emotional Needs
Accepta

In [ ]:
_ID = test_data[i]['subID'] + '###' + _author
_closest_subIDs = [situIndex_to_subID[elem[0]] for elem in ID_to_closest[_ID]]

In [264]:
situIndex_to_subID[5596]

'bfo3l0'

In [256]:
ID_to_closest['h0eavk###iluvcats17']

[(8923, np.float64(0.5453128679496254)),
 (5587, np.float64(0.5679981941844443)),
 (10422, np.float64(0.5752216769600275)),
 (17995, np.float64(0.579622500413684)),
 (5596, np.float64(0.5900049859159866)),
 (17288, np.float64(0.5951953101052884)),
 (17951, np.float64(0.602581949173784)),
 (5812, np.float64(0.6133462192589)),
 (305, np.float64(0.6160237454539)),
 (9084, np.float64(0.6176272404050239)),
 (9499, np.float64(0.618777664187131)),
 (16904, np.float64(0.619281000533626)),
 (8940, np.float64(0.6196888189671742)),
 (17017, np.float64(0.6197051671319359)),
 (10144, np.float64(0.6206748651861085)),
 (9618, np.float64(0.6214740093216748)),
 (14806, np.float64(0.6227577413297641)),
 (10262, np.float64(0.6240368787551943)),
 (15917, np.float64(0.6244519846123708)),
 (14962, np.float64(0.6247278982160006)),
 (5700, np.float64(0.6249992981023544)),
 (5705, np.float64(0.6260154820836794)),
 (9125, np.float64(0.62680592267759)),
 (15136, np.float64(0.6279627179881649)),
 (9953, np.float6

In [262]:
df_ext[df_ext['subID']=='jb0o3r']

,Unnamed: 0,subID,title,selftext,commAuthor,comment,judgment
5402,5402,jb0o3r,saying I'm a single mom?,So I (29f) have a wonderful 8 year old daughte...,iluvcats17,I would not consider you a single mom since yo...,0
5407,5407,jb0o3r,saying I'm a single mom?,So I (29f) have a wonderful 8 year old daughte...,seba_make,You’re a mother and not in a relationship. So ...,1
5428,5428,jb0o3r,saying I'm a single mom?,So I (29f) have a wonderful 8 year old daughte...,Dachshundmom5,you are a single mother. That is true and has ...,1
